In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import RandomizedSearchCV
import tensorflow as tf
import keras.metrics
import numpy as np
import joblib
from scipy.stats import reciprocal


In [8]:
def return_model(n_hidden = 1 , n_neurons=30, learning_rate = 0.01, input_shape = X_train.shape[1:]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape = input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu",  activity_regularizer=l1(0.01)))
        model.add(Dropout(0.3))

    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate = learning_rate)
    model.compile(optimizer=optimizer, loss= 'mse')
    return model 

In [3]:
sheet = pd.read_excel(r"test1.xlsx")
print(sheet.info())

sheet = sheet.dropna()
features = sheet.iloc[:,1:4]
result = sheet.iloc[:,4:]
print(features)
X_train,X_test, y_train, y_test = train_test_split(features,result.iloc[:,3], test_size = 0.05)
X_train,X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2102 entries, 0 to 2101
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   OndernemingID     2102 non-null   int64  
 1   AantalWerknemers  1894 non-null   float64
 2   Omzet             1725 non-null   float64
 3   TotaalActiva      2055 non-null   float64
 4   duurzaamheidrank  2102 non-null   float64
 5   governancerank    2102 non-null   float64
 6   environmentdrank  2102 non-null   float64
 7   TotalRank         2102 non-null   float64
dtypes: float64(7), int64(1)
memory usage: 131.5 KB
None
      AantalWerknemers      Omzet  TotaalActiva
0                 22.0   458778.0    41156846.0
1                 65.0   123111.0    24339573.0
2                133.0   113602.0    21680719.0
3                114.0   120825.0    32786485.0
4                167.0   128556.0    35031532.0
...                ...        ...           ...
2024              11.0  4925783

In [9]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
X_valid = scaler.fit_transform(X_val)
print(X_train.shape[1:])
batch_size = [10, 20, 40, 60, 80, 100, 1000]
epochs = [10, 50, 100, 200, 500]
param_grid = dict(batch_size=batch_size, epochs=epochs)

(3,)


In [10]:
model = KerasRegressor(return_model)
param_dist = {
    "n_hidden":[0,1,2,3,4,5,6],
    "n_neurons":np.arange(1,250),
    "learning_rate":reciprocal(3e-5, 3e-2)
}
print(y_test)
print(X_test)
print(y_train)
rnd_mod = RandomizedSearchCV(model, param_distributions=param_dist, n_iter= 50, cv=5)
rnd_mod.fit(X_train, y_train, epochs= 20, validation_data=(X_test, y_test))

joblib.dump(rnd_mod, 'model.pkl')

535     0.000000
516     0.444444
1953    0.000000
391     0.411765
452     0.000000
          ...   
863     0.000000
1660    0.000000
1879    0.000000
1508    0.166667
17      0.000000
Name: TotalRank, Length: 85, dtype: float64
[[-8.49661588e-01 -5.58358190e-01  1.07866544e+00]
 [ 4.46092392e-01 -5.57652541e-01 -2.37515242e-01]
 [-7.35998958e-01 -3.33592345e-02 -2.19346469e-01]
 [ 3.46951834e+00 -5.57903982e-01  2.56530957e-01]
 [-5.76871276e-01 -5.57967112e-01 -1.75509172e-01]
 [-7.13266432e-01 -5.58628050e-01  2.10637772e-01]
 [-7.58731484e-01 -5.58735531e-01 -3.27038681e-01]
 [-5.54138750e-01 -5.58098260e-01 -1.75766020e-01]
 [-3.12906534e-02 -5.54846667e-01 -5.45363967e-02]
 [ 1.28719585e+00 -5.56891634e-01 -4.72475338e-02]
 [-7.67557053e-02 -5.58412831e-01 -2.79540130e-01]
 [-7.35998958e-01  1.18722633e-01 -2.51037916e-01]
 [ 1.81004395e+00 -5.58195760e-01  1.83627517e+00]
 [-9.17859166e-01  4.65343294e+00 -2.49529738e-01]
 [ 3.69069245e-02 -5.52517846e-01 -1.73921459e-01]
 [-3

C:\Users\Benjamin\AppData\Local\Temp\ipykernel_9212\301836247.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasRegressor(return_model)


37/37 [==============================] - 0s 3ms/step - loss: 0.6149 - val_loss: 0.6655
Epoch 2/20
37/37 [==============================] - 0s 721us/step - loss: 0.4621 - val_loss: 0.4928
Epoch 3/20
37/37 [==============================] - 0s 694us/step - loss: 0.3548 - val_loss: 0.3706
Epoch 4/20
37/37 [==============================] - 0s 694us/step - loss: 0.2778 - val_loss: 0.2835
Epoch 5/20
37/37 [==============================] - 0s 694us/step - loss: 0.2237 - val_loss: 0.2215
Epoch 6/20
37/37 [==============================] - 0s 694us/step - loss: 0.1853 - val_loss: 0.1775
Epoch 7/20
37/37 [==============================] - 0s 694us/step - loss: 0.1574 - val_loss: 0.1459
Epoch 8/20
37/37 [==============================] - 0s 709us/step - loss: 0.1374 - val_loss: 0.1232
Epoch 9/20
37/37 [==============================] - 0s 694us/step - loss: 0.1232 - val_loss: 0.1069
Epoch 10/20
37/37 [==============================] - 0s 694us/step - loss: 0.1131 - val_loss: 0.0952
Epoch 11/20


Epoch 19/20
37/37 [==============================] - 0s 694us/step - loss: 0.1301 - val_loss: 0.0977
Epoch 20/20
10/10 [==============================] - 0s 445us/step - loss: 0.1024
Epoch 1/20
37/37 [==============================] - 0s 2ms/step - loss: 1.7750 - val_loss: 1.5116
Epoch 2/20
37/37 [==============================] - 0s 667us/step - loss: 1.4637 - val_loss: 1.2458
Epoch 3/20
37/37 [==============================] - 0s 694us/step - loss: 1.2065 - val_loss: 1.0330
Epoch 4/20
37/37 [==============================] - 0s 694us/step - loss: 1.0016 - val_loss: 0.8581
Epoch 5/20
37/37 [==============================] - 0s 694us/step - loss: 0.8329 - val_loss: 0.7158
Epoch 6/20
37/37 [==============================] - 0s 694us/step - loss: 0.6971 - val_loss: 0.5979
Epoch 7/20
37/37 [==============================] - 0s 667us/step - loss: 0.5848 - val_loss: 0.5029
Epoch 8/20
37/37 [==============================] - 0s 667us/step - loss: 0.4948 - val_loss: 0.4249
Epoch 9/20
37/37 [=

Epoch 18/20
37/37 [==============================] - 0s 667us/step - loss: 0.0895 - val_loss: 0.0641
Epoch 19/20
37/37 [==============================] - 0s 667us/step - loss: 0.0895 - val_loss: 0.0641
Epoch 20/20
10/10 [==============================] - 0s 556us/step - loss: 0.0904
Epoch 1/20
37/37 [==============================] - 0s 2ms/step - loss: 1.7968 - val_loss: 1.0226
Epoch 2/20
37/37 [==============================] - 0s 667us/step - loss: 0.6614 - val_loss: 0.3772
Epoch 3/20
37/37 [==============================] - 0s 667us/step - loss: 0.2745 - val_loss: 0.1705
Epoch 4/20
37/37 [==============================] - 0s 694us/step - loss: 0.1537 - val_loss: 0.0993
Epoch 5/20
37/37 [==============================] - 0s 667us/step - loss: 0.1122 - val_loss: 0.0762
Epoch 6/20
37/37 [==============================] - 0s 694us/step - loss: 0.0984 - val_loss: 0.0679
Epoch 7/20
37/37 [==============================] - 0s 694us/step - loss: 0.0935 - val_loss: 0.0653
Epoch 8/20
37/37 [

...

Epoch 14/20
37/37 [==============================] - 0s 750us/step - loss: 0.0913 - val_loss: 0.0647
Epoch 15/20
37/37 [==============================] - 0s 694us/step - loss: 0.0911 - val_loss: 0.0637
Epoch 16/20
37/37 [==============================] - 0s 694us/step - loss: 0.0912 - val_loss: 0.0634
Epoch 17/20
37/37 [==============================] - 0s 667us/step - loss: 0.0912 - val_loss: 0.0633
Epoch 18/20
37/37 [==============================] - 0s 667us/step - loss: 0.0911 - val_loss: 0.0642
Epoch 19/20
37/37 [==============================] - 0s 667us/step - loss: 0.0911 - val_loss: 0.0645
Epoch 20/20
10/10 [==============================] - 0s 382us/step - loss: 0.0852
Epoch 1/20
37/37 [==============================] - 0s 2ms/step - loss: 0.6738 - val_loss: 0.0778
Epoch 2/20
37/37 [==============================] - 0s 694us/step - loss: 0.0991 - val_loss: 0.0640
Epoch 3/20
37/37 [==============================] - 0s 694us/step - loss: 0.0919 - val_loss: 0.0646
Epoch 4/20
37/

Epoch 13/20
37/37 [==============================] - 0s 712us/step - loss: 0.0896 - val_loss: 0.0639
Epoch 14/20
37/37 [==============================] - 0s 694us/step - loss: 0.0896 - val_loss: 0.0638
Epoch 15/20
37/37 [==============================] - 0s 694us/step - loss: 0.0897 - val_loss: 0.0640
Epoch 16/20
37/37 [==============================] - 0s 694us/step - loss: 0.0896 - val_loss: 0.0642
Epoch 17/20
37/37 [==============================] - 0s 667us/step - loss: 0.0897 - val_loss: 0.0644
Epoch 18/20
37/37 [==============================] - 0s 694us/step - loss: 0.0897 - val_loss: 0.0650
Epoch 19/20
37/37 [==============================] - 0s 681us/step - loss: 0.0897 - val_loss: 0.0641
Epoch 20/20
10/10 [==============================] - 0s 444us/step - loss: 0.0907
Epoch 1/20
37/37 [==============================] - 0s 2ms/step - loss: 1.0889 - val_loss: 0.2762
Epoch 2/20
37/37 [==============================] - 0s 667us/step - loss: 0.1825 - val_loss: 0.0896
Epoch 3/20
37

C:\Users\Benjamin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
215 fits failed out of a total of 250.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
215 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Benjamin\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Benjamin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\wrappers\scikit_learn.py", line 164, in fit
    self.model = self.build_fn(**self.filter_sk_params(self.build_fn))
  File "C:\Users\Benjamin\AppData\Lo

46/46 [==============================] - 0s 689us/step - loss: 0.3856 - val_loss: 0.1754
Epoch 3/20
46/46 [==============================] - 0s 662us/step - loss: 0.1792 - val_loss: 0.0936
Epoch 4/20
46/46 [==============================] - 0s 622us/step - loss: 0.1168 - val_loss: 0.0707
Epoch 5/20
46/46 [==============================] - 0s 622us/step - loss: 0.0979 - val_loss: 0.0648
Epoch 6/20
46/46 [==============================] - 0s 600us/step - loss: 0.0920 - val_loss: 0.0636
Epoch 7/20
46/46 [==============================] - 0s 644us/step - loss: 0.0903 - val_loss: 0.0634
Epoch 8/20
46/46 [==============================] - 0s 622us/step - loss: 0.0898 - val_loss: 0.0634
Epoch 9/20
46/46 [==============================] - 0s 644us/step - loss: 0.0896 - val_loss: 0.0635
Epoch 10/20
46/46 [==============================] - 0s 632us/step - loss: 0.0896 - val_loss: 0.0636
Epoch 11/20
46/46 [==============================] - 0s 622us/step - loss: 0.0896 - val_loss: 0.0636
Epoch 12/

['model.pkl']

In [11]:
print(rnd_mod.predict(X_val))
print(y_val)


6/6 [==============================] - 0s 544us/step
[-8.66493203e+04 -3.95620273e+04  1.22460961e+05  2.07558953e+05
  1.36032016e+05  3.00025688e+05 -8.74965703e+04 -2.53256016e+04
  6.95962312e+05  1.87209578e+05  2.18681328e+05 -5.52905391e+04
 -8.98580547e+04 -4.21034297e+04 -2.67558223e+04  5.75128938e+05
 -5.57723672e+04  2.50894250e+05 -7.34323281e+04 -3.57201758e+04
 -9.55987812e+04  2.41408719e+05 -4.97680625e+05  3.68237625e+05
 -2.27769980e+04 -6.01422125e+05 -4.76472031e+04 -5.20679375e+04
 -4.94622539e+04 -7.64905078e+04  1.46789797e+05  8.88500781e+04
 -8.00396641e+04 -1.70621465e+04 -6.22740156e+04 -1.15953688e+05
 -9.59864746e+03  1.50915453e+05  2.12072250e+05 -3.96322583e+03
  3.23160000e+05 -8.04558281e+04 -3.95569000e+05 -1.63437900e+04
  3.41104906e+05  9.57476328e+04 -1.67427969e+04 -9.37517344e+04
 -1.35401641e+05 -2.58003484e+05 -3.52055820e+04  1.47076484e+05
 -8.35195312e+04 -4.29667266e+04  8.87548312e+05 -2.19762516e+05
 -6.32499258e+04  1.32922156e+05  5.2